In [1]:
import os
import torch
import torch.nn as nn
from torch import cuda
from torchvision import models
import albumentations as A
from modules.model import create_model

import numpy as np
import pandas as pd
from tqdm import tqdm
from argparse import ArgumentParser

from dataloader import CustomDataLoader, do_transform, collate_fn

In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'
num_workers = 4
batch_size = 8
resize = 256

save_dir = '/opt/ml/soft_ensemble'
data_dir = '/opt/ml/input/data'
model_paths = ['/opt/ml/trained_models/Final_fold_0/ep89_0.7374.pt', '/opt/ml/trained_models/Final_fold_0/ep113_0.7191.pt', '/opt/ml/trained_models/Final_fold_0/ep130_0.7177.pt']
use_models = ['mvtb4_unet', 'mvtb4_unet', 'mvtb4_unet']

In [6]:
def test_with_maps(model_paths, use_models, batch_size=batch_size, num_workers=num_workers, resize=resize, device=device, data_dir=data_dir, save_dir=save_dir):
    # load models
    models = []
    for model_path, use_model in zip(model_paths, use_models):
        model = create_model(use_model)
        checkpoint = torch.load(model_path, map_location=device)
        state_dict = checkpoint.state_dict()
        model.load_state_dict(state_dict)
        model = model.to(device)
        model.eval()
        models.append(model)

    # data loader
    test_dataset = CustomDataLoader(data_dir=os.path.join(data_dir, 'test.json'),
                                   mode='test',
                                   transform=do_transform(mode='test'),
                                   data_path=data_dir)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                batch_size=batch_size,
                                                num_workers=num_workers,
                                                collate_fn=collate_fn)
    size = resize
    transform = A.Compose([A.Resize(size, size)])
    print('\nStart prediction.')
    
    # Predict
    
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        for step, (imgs, image_infos) in enumerate(tqdm(test_loader)):
            # inference (512 x 512)
            outs = []
            for model in models:
                outs.append(model(torch.stack(imgs).to(device)))     # model(torch.stack(imgs).to(device)): torch.Size([N_batch, 11, 512, 512])
            outs = torch.stack(outs)
            outs = torch.mean(outs, axis=0)
            oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)
                
            oms = np.array(temp_mask)
            
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])

    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    print("Start Inference")
    # sample_submisson.csv 열기
    submission = pd.read_csv('/opt/ml/input/code/submission/sample_submission.csv', index_col=None)

    # PredictionString 대입
    for file_name, string in zip(file_names, preds_array):
        submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                    ignore_index=True)

    # submission.csv로 저장
    submission.to_csv(os.path.join(save_dir, 'submission.csv'), index=False)
    print("End Inference")

In [7]:
test_with_maps(model_paths, use_models)

  0%|          | 0/103 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Start prediction.


100%|██████████| 103/103 [08:39<00:00,  5.04s/it]


End prediction.
Start Inference
End Inference
